In [1]:
%reload_ext autoreload
%autoreload 2
from aldiscore.prediction.extractor import FeatureExtractor
from aldiscore.prediction import utils
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
from pathlib import Path
from Bio import SeqIO
from aldiscore import ROOT, get_from_config
import numpy as np
import pandas as pd

Trials:
- tc_base vs tc_different_statistics!
- tc_base vs tc_dist_abs vs tc_dist_scaled vs tc_dist_log
- tc_base + (tc_dist_abs vs tc_dist_scaled vs tc_dist_log)
- tc_base vs tc_50 vs tc_100

TODOs:
- Try new/better aggregation statistics
- Add remaining PSA features
- Look into simulation paper again


In [4]:
TEST_FILE_AA = Path("data/aa.fasta")
TEST_FILE_DNA = Path("data/dna.fasta")
INTRON_FILE = Path("../../data/backup/alns/GALGAL_R00007_intron1.fa")

sequences = list(SeqIO.parse(INTRON_FILE, "fasta"))[:10]
for seq_record in sequences:
    seq_record.seq = Seq(str(seq_record.seq).replace("-", ""))

print("k =", len(sequences))

seq_lens = [len(seq) for seq in sequences]
# print(sequences[0].seq[:100])
# print(sequences[29].seq[:100])
extractor = FeatureExtractor(sequences, track_perf=True)
feats = extractor.compute()
extractor._perf_dict

# feats.loc[:, feats.columns.str.contains("chi-square")]
# feats.loc[:, feats.columns.str.contains("tc_")]

k = 10


{'_init_basics': 0.0025829635560512543,
 '_init_char_dists': 0.0020705200731754303,
 '_init_psa': 4.9907663986086845,
 '_init_psa_index_map': 0.0028416067361831665,
 '_data_type': 2.6598572731018066e-06,
 '_num_sequences': 2.2388994693756104e-06,
 '_sequence_length': 0.00027086958289146423,
 '_sequence_length_ratio': 1.2382864952087402e-05,
 '_lower_bound_gap_percentage': 1.7035752534866333e-05,
 '_sequence_entropy': 0.00016639754176139832,
 '_char_js_divergence': 0.00028632208704948425,
 '_homopolymer_js_divergence': 0.018783051520586014,
 '_transitive_consistency': 0.041419535875320435,
 '_psa_basic_features': 0.0014189593493938446,
 '_psa_gap_length': 0.012194707989692688,
 '_kmer_similarity': 0.18769316747784615}

In [6]:
counts, lens = utils.repeat_distributions(sequences)

print(counts.shape)

import seaborn as sn

sn.histplot(counts.T, element="poly")

(4, 43)


KeyboardInterrupt: 

In [ ]:
import pyfastx

for item in pyfastx.Fastx(TEST_FILE_DNA, uppercase=True):
    print(item)

('hmgl_trybr', 'AAGAAGGACTCTAATGCTCCGAAGCGGGCTATGACTTCTTTTATGTTCTTCTCATCGGACTTCCGAAGCAAGCATAGCGACTTATCGATAGTAGAGATGAGTAAGGCGGCAGGGGCAGCCTGGAAGGAGCTGGGGCCCGAGGAGCGCAAGGTGTATGAGGAAATGGCAGAGAAGGATAAAGAACGCTATAAGCGTGAGATG')
('hmgt_mouse', 'AAGCCTAAACGTCCTCGAAGTGCTTACAACATCTATGTCTCAGAAAGTTTTCAGGAGGCGAAAGATGACTCCGCGCAGGGCAAGTTGAAACTCGTTAATGAAGCCTGGAAAAATCTAAGTCCTGAGGAGAAGCAAGCCTATATCCAGTTGGCAAAAGACGACAGAATACGCTATGACAATGAAATGAAGTCATGGGAAGAGCAAATGGCTGAA')
('hmgb_chite', 'GCGGATAAACCTAAACGGCCATTAAGTGCGTACATGCTATGGCTAAATTCCGCCCGAGAATCTATTAAAAGAGAAAATCCTGATTTTAAAGTGACCGAGGTAGCTAAAAAGGGTGGTGAATTGTGGCGTGGTCTAAAGGATAAGTCGGAGTGGGAGGCAAAGGCTGCCACCGCTAAACAAAATTACATCAGGGCGCTTCAGGAATATGAGCGGAATGGCGGA')
('hmgl_wheat', 'GACCCTAATAAACCAAAACGCGCGCCCTCAGCTTTCTTCGTGTTTATGGGGGAGTTCAGAGAAGAATTTAAACAAAAAAATCCCAAGAACAAGTCCGTCGCCGCTGTTGGTAAGGCTGCGGGGGAACGATGGAAGAGTCTTTCAGAGAGCGAAAAGGCGCCGTACGTGGCCAAAGCAAATAAATTAAAAGGAGAGTATAATAAGGCGATTGCCGCATACAATAAGGGCGAATCTGCA')


In [ ]:
from itertools import groupby
from collections import defaultdict
import pytrf, pyfastx
from functools import partial

# from pytrf import d


def tandem_repeats(sequences):
    # parse input fasta sequence
    for seq_record in sequences:
        seq = str(seq_record.seq).upper()
        name = seq_record.id
        # get the first sequence from fasta
        # feed sequence to pytrf
        # the fastest way to get all SSRs from sequence
        ssrs = pytrf.STRFinder(name, seq).as_list()
        print(ssrs)


from aldiscore.prediction import utils

count_arr, len_arr = tandem_repeats(sequences)
len_arr

[('Acanthisitta_chloris', 509, 522, 'A', 1, 14, 14)]
[]
[]
[]
[]
[]


TypeError: cannot unpack non-iterable NoneType object

In [5]:
perf_dir = ROOT / "logs" / "perf"
perf_file = sorted(os.listdir(perf_dir))[-1]
print(perf_file)
perf_df = pd.read_parquet(perf_dir / perf_file)
perf_df
# perf_df.sum(axis=1).groupby("source").median()

2025-10-02_13-43-37.parquet


_init_basics  _init_char_dists  \
source       dataset                                                         
bralibase_k5 HIV_FE.apsi-86.sci-94.no-5         0.113276          0.000500   
             HCV_SLIV.apsi-86.sci-100.no-1      0.081990          0.000302   
             HIV_FE.apsi-92.sci-94.no-1         0.088590          0.000486   
             HIV_FE.apsi-87.sci-90.no-2         0.092224          0.000393   
             HIV_GSL3.apsi-88.sci-77.no-2       0.060637          0.000392   
...                                                  ...               ...   
treebase_v1  20196_21.phy                       0.004679          0.002546   
             14688_30.phy                       0.007059          0.000350   
             18202_8.phy                        0.017448          0.029228   
             16461_8.phy                        0.016802          0.018810   
             11746_2.phy                        0.008457          0.013142   

                                            _init_psa  _init_psa_index_map  \
source       dataset                                                         
bralibase_k5 HIV_FE.apsi-86.sci-94.no-5      0.013520             0.000534   
             HCV_SLIV.apsi-86.sci-100.no-1   0.013632             0.000607   
             HIV_FE.apsi-92.sci-94.no-1      0.013638             0.000589   
             HIV_FE.apsi-87.sci-90.no-2      0.013503             0.000537   
             HIV_GSL3.apsi-88.sci-77.no-2    0.014303             0.000579   
...                                               ...                  ...   
treebase_v1  20196_21.phy                    1.147992             0.031682   
             14688_30.phy                    0.021165             0.000889   
             18202_8.phy                    52.351813             0.069025   
             16461_8.phy                    26.813274             0.067993   
             11746_2.phy                    38.343293             0.055424   

                                            _data_type  _num_sequences  \
source       dataset                                                     
bralibase_k5 HIV_FE.apsi-86.sci-94.no-5       0.000009        0.000007   
             HCV_SLIV.apsi-86.sci-100.no-1    0.000010        0.000009   
             HIV_FE.apsi-92.sci-94.no-1       0.000011        0.000009   
             HIV_FE.apsi-87.sci-90.no-2       0.000013        0.000008   
             HIV_GSL3.apsi-88.sci-77.no-2     0.000010        0.000008   
...                                                ...             ...   
treebase_v1  20196_21.phy                     0.000006        0.000003   
             14688_30.phy                     0.000005        0.000003   
             18202_8.phy                      0.000006        0.000003   
             16461_8.phy                      0.000006        0.000003   
             11746_2.phy                      0.000006        0.000003   

                                            _sequence_length  \
source       dataset                                           
bralibase_k5 HIV_FE.apsi-86.sci-94.no-5             0.003386   
             HCV_SLIV.apsi-86.sci-100.no-1          0.003383   
             HIV_FE.apsi-92.sci-94.no-1             0.003171   
             HIV_FE.apsi-87.sci-90.no-2             0.003521   
             HIV_GSL3.apsi-88.sci-77.no-2           0.002693   
...                                                      ...   
treebase_v1  20196_21.phy                           0.000522   
             14688_30.phy                           0.000508   
             18202_8.phy                            0.000512   
             16461_8.phy                            0.000474   
             11746_2.phy                            0.000505   

                                            _sequence_length_ratio  \
source       dataset                                                 
bralibase_k5 HIV_FE.apsi-86.sci-94.no-5                   0.000018   
        

In [ ]:
from tqdm import tqdm

source = "treebase_v1"
data_dir = Path("/hits/fast/cme/bodynems/data/paper") / source
datasets = list(
    filter(lambda name: os.path.isdir(data_dir / name), os.listdir(data_dir))
)

feats_dict = {}
for i, dataset in tqdm(enumerate(datasets)):
    # if dataset != "25479_0.phy":
    #     continue
    seqs = list(SeqIO.parse(data_dir / dataset / "sequences.fasta", format="fasta"))
    feats = FeatureExtractor(seqs).compute()
    feats_dict[(source, dataset)] = feats
    if i >= 24:
        break
feat_df = pd.concat(feats_dict.values(), axis=0, ignore_index=True)
feat_df.index = pd.MultiIndex.from_tuples(
    feats_dict.keys(), names=["source", "dataset"]
)

# feat_df.info()

In [ ]:
import parasail
from Bio.Seq import Seq
from Bio import SeqIO
import itertools
import random
from aldiscore.scoring.encoding import encode_positions
from aldiscore.enums.enums import PositionalEncodingEnum as PSE
from collections import defaultdict

seq_path = Path("/hits/fast/cme/bodynems/data/paper/prefab4/1cvl_1tca/sequences.fasta")
sequences = list(SeqIO.parse(seq_path, format="fasta"))
print(len(sequences))
print(sequences)
GAP_CHAR = "-"
# sequences = [
#     SeqRecord(Seq("ACCCA")),
#     SeqRecord(Seq("ACAGTAAACCA")),
#     SeqRecord(Seq("ACGGTACATTA")),
#     SeqRecord(Seq("TTCTTAG")),
#     SeqRecord(Seq("GCTACTATTA")),
#     SeqRecord(Seq("TTAATGGGGGGA")),
#     SeqRecord(Seq("ATTTCTA")),
#     SeqRecord(Seq("ATTTATTAAACCA")),
#     SeqRecord(Seq("ATTTGTTACATTA")),
#     SeqRecord(Seq("TTTTCTTAG")),
#     SeqRecord(Seq("GTTTTTCTATTA")),
#     SeqRecord(Seq("TTTTATTGGGGGGA")),
# ]
extractor = FeatureExtractor(sequences)
extractor._init_cache()
out = extractor.compute()
print(len(out.columns))
print(len(out.columns[out.columns.str.contains("tc_")]))
for i, col in enumerate(out.columns):
    print(col, end=" | ")
    if (i + 1) % 6 == 0:
        print()
# idx_map = extractor._cache[extractor._PSA_INDEX_MAP]
# for idx_a in idx_map:
#     print(idx_map[idx_a])

# print(groups)
# for idx_a in aligns:
#     print(idx_a)
#     for idx_b in aligns[idx_a]:
#         print("  " + str(idx_b))